In [159]:
import numpy as np
import pandas as pd

data = pd.read_csv("C:\\Users\\SIDDU\\OneDrive\\Desktop\\work\\selflearnml\\practice\\adult.data",na_values=' ?')
data.columns = ["age","workclass","fnlwgt","education","education-num","marital-status",
    "occupation","relationship","race","sex","capital-gain","capital-loss","hours-per-week",
    "native-country","Salary"]
# print(data.info())
# data = data.dropna(axis=0)
# print(data.shape)

# print(pd.DataFrame(data[['workclass']].value_counts()))

                       0
workclass               
 Private           22696
 Self-emp-not-inc   2541
 Local-gov          2093
 State-gov          1297
 Self-emp-inc       1116
 Federal-gov         960
 Without-pay          14
 Never-worked          7


In [160]:
X = data.drop(["Salary"],axis=1)
y = data["Salary"]

In [161]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, stratify=y, random_state=10)

In [162]:
from sklearn.preprocessing import LabelEncoder
lb = LabelEncoder()
y_train_encoded = pd.Series(lb.fit_transform(y_train))
y_test_encoded = pd.Series(lb.transform(y_test))
y_train_encoded.name = y_train.name
y_test_encoded.name = y_test.name
print(y_test_encoded.head())

0    0
1    1
2    0
3    1
4    0
Name: Salary, dtype: int32


In [163]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(strategy="most_frequent")
X_train_imputed = pd.DataFrame(imp.fit_transform(X_train))
X_test_imputed = pd.DataFrame(imp.transform(X_test))
X_train_imputed.columns = X_train.columns
X_test_imputed.columns = X_test.columns
print(X_train_imputed)

      age      workclass  fnlwgt      education education-num  \
0      46        Private  503923        HS-grad             9   
1      36        Private   48972   Some-college            10   
2      43        Private  145441      Bachelors            13   
3      65   Self-emp-inc  115922      Bachelors            13   
4      22        Private  268145      Bachelors            13   
...    ..            ...     ...            ...           ...   
26043  51        Private  126010        HS-grad             9   
26044  43        Private  133584   Some-college            10   
26045  54        Private  175912           10th             6   
26046  72    Federal-gov  217864        HS-grad             9   
26047  25      State-gov  328697   Some-college            10   

            marital-status          occupation     relationship    race  \
0       Married-civ-spouse   Machine-op-inspct          Husband   Black   
1                 Divorced        Adm-clerical        Unmarried   Whi

In [164]:
OHEfeatures = ["workclass","marital-status","occupation","relationship","race","sex","native-country"]
from sklearn.preprocessing import OneHotEncoder
oh = OneHotEncoder(sparse=False,handle_unknown='ignore')
X_train_encoded = pd.DataFrame(oh.fit_transform(X_train_imputed[OHEfeatures]))
X_test_encoded = pd.DataFrame(oh.transform(X_test_imputed[OHEfeatures]))


In [165]:
edu = [' Preschool',' 1st-4th',' 5th-6th',' 7th-8th',' 9th',' 10th',' 11th',' 12th',' HS-grad' ,' Prof-school',' Some-college',' Assoc-acdm',' Assoc-voc',' Bachelors',' Masters',' Doctorate']
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories=[edu])
X_train_imputed[["education"]] = pd.DataFrame(oe.fit_transform(X_train_imputed[["education"]]))
X_test_imputed[["education"]] = pd.DataFrame(oe.transform(X_test_imputed[["education"]]))
print(X_train_imputed.education)

0         8.0
1        10.0
2        13.0
3        13.0
4        13.0
         ... 
26043     8.0
26044    10.0
26045     5.0
26046     8.0
26047    10.0
Name: education, Length: 26048, dtype: float64


In [172]:
X_train_final = X_train_imputed.drop(OHEfeatures, axis=1).merge(X_train_encoded,left_index=True, right_index=True)
X_test_final = X_test_imputed.drop(OHEfeatures, axis=1).merge(X_test_encoded,left_index=True, right_index=True)
# print(X_train_final[['capital-gain']].value_counts())

capital-gain
0               23875
15024             293
7688              229
7298              193
99999             118
                ...  
2538                1
4931                1
7978                1
7896                1
1639                1
Length: 117, dtype: int64


In [167]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=21)
model.fit(X_train_final,y_train_encoded)
predictions = model.predict(X_test_final)
from sklearn.metrics import accuracy_score
print(accuracy_score(predictions,y_test_encoded))
print(pd.Series(lb.inverse_transform(predictions),name='Salary'))

0.7977579852579852
0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
6507     <=50K
6508      >50K
6509     <=50K
6510     <=50K
6511     <=50K
Name: Salary, Length: 6512, dtype: object


In [168]:
from sklearn.svm import SVC
model = SVC()
model.fit(X_train_final,y_train_encoded)
predictions2 = model.predict(X_test_final)
print(accuracy_score(predictions2, y_test_encoded))
print(pd.Series(lb.inverse_transform(predictions),name='Salary'))

0.7936117936117936
0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
6507     <=50K
6508      >50K
6509     <=50K
6510     <=50K
6511     <=50K
Name: Salary, Length: 6512, dtype: object


In [174]:
# from sklearn.preprocessing import StandardScaler
# sc_X = StandardScaler()
# X_train = sc_X.fit_transform(X_train)
# X_test = sc_X.fit_transform(X_test)

In [176]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(X_train_final,y_train_encoded)
predictions2 = model.predict(X_test_final)
print(accuracy_score(predictions2, y_test_encoded))
print(pd.Series(lb.inverse_transform(predictions),name='Salary'))

0.8155712530712531
0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
6507     <=50K
6508      >50K
6509     <=50K
6510     <=50K
6511     <=50K
Name: Salary, Length: 6512, dtype: object


In [183]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=500)
model.fit(X_train_final,y_train_encoded)
predictions2 = model.predict(X_test_final)
print(accuracy_score(predictions2, y_test_encoded))
print(pd.Series(lb.inverse_transform(predictions),name='Salary'))

0.8576474201474201
0        <=50K
1        <=50K
2        <=50K
3        <=50K
4        <=50K
         ...  
6507     <=50K
6508      >50K
6509     <=50K
6510     <=50K
6511     <=50K
Name: Salary, Length: 6512, dtype: object
